# Rolling FEVD estimation
## Imports

In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import pandas as pd
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mno

import networkx as nx
import datetime as dt

from sklearn.model_selection import GridSearchCV

import sys
sys.path.append('../')

import src

sys.path.append('../../kungfu/')
import kungfu as kf

/home/fbrun/.local/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
# # RUN THIS TO CREATE FILSYSTEM STRUCTURE

# import os
# path = '../data/estimated/monthly/'
# for year in range(1994, 2021):
#     os.mkdir(path+str(year))
#     for month in range(1,13):
#         os.mkdir(path+str(year)+'/'+str(month))

## Setup

In [4]:
var_grid = {'alpha': np.geomspace(1e-5, 1e0, 11),
            'lambdau': np.geomspace(1e-2, 1e2, 13),
            #'gamma': np.geomspace(1e-2, 1e2, 15),
            }

# cov_grid = {'delta': np.geomspace(0.5, 1, 11),
#             'eta': np.linspace(0, 2, 13)}

cov_grid = {'alpha': np.geomspace(1e-2, 1e0, 25)}

horizon = 21

In [5]:
desc_all = pd.DataFrame()
network_data_all = pd.DataFrame()

In [6]:
desc_all = pd.read_csv('../data/estimated/estimation_statistics.csv')
desc_all.index = desc_all['Unnamed: 0'].values
desc_all = desc_all.drop(columns=['Unnamed: 0'])

network_data_all = pd.read_csv('../data/estimated/network_data.csv')
network_data_all = network_data_all.set_index(['sampling_date', 'permno'])

#### Trial Single Estimation

In [10]:
year = 2020
month = 12

In [11]:
var_grid = {'alpha': [1e-2],
            'lambdau': [1e-1],
            #'gamma': np.geomspace(1e-2, 1e2, 15),
            }

# cov_grid = {'delta': np.geomspace(0.5, 1, 11),
#             'eta': np.linspace(0, 2, 13)}

cov_grid = {'alpha': [1e-1]}

horizon = 21

In [12]:
%%time
preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 1 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    3.2s finished


Fitting 12 folds for each of 1 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   10.8s finished
/home/fbrun/.local/lib/python3.6/site-packages/sklearn/covariance/_graph_lasso.py:266: ConvergenceWarning: graphical_lasso: did not converge after 200 iteration: dual gap: -2.734e-03
  % (max_iter, d_gap), ConvergenceWarning)


CPU times: user 30.1 s, sys: 11.4 s, total: 41.5 s
Wall time: 1min 51s


#### Rolling Estimation

In [7]:
%%time
for year in range(1994, 2001):
    for month in range(1, 13): #[3, 6, 9, 12]
        preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)
    
        # append data
        desc_all = desc_all.append(desc.rename(preprocessed_data.index[-1]).to_frame().T)
        network_data.index = pd.MultiIndex.from_product([[preprocessed_data.index[-1]],network_data.index], names=['sampling_date', 'permno'])
        network_data_all = network_data_all.append(network_data)

        print('DONE WITH {}/{}\n '.format(month, year))
    
        desc_all.to_csv('../data/estimated/estimation_statistics.csv')
        network_data_all.to_csv('../data/estimated/network_data.csv')

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   34.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.6min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   49.2s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   57.0s finished


DONE WITH 1/1994
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


...

DONE WITH 11/2000
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.8min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.4min finished


DONE WITH 12/2000
 
CPU times: user 40min 52s, sys: 10min 6s, total: 50min 58s
Wall time: 10h 39min 52s


In [7]:
%%time
for year in range(2001, 2006):
    for month in range(1, 13): #[3, 6, 9, 12]
        preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)
    
        # append data
        desc_all = desc_all.append(desc.rename(preprocessed_data.index[-1]).to_frame().T)
        network_data.index = pd.MultiIndex.from_product([[preprocessed_data.index[-1]],network_data.index], names=['sampling_date', 'permno'])
        network_data_all = network_data_all.append(network_data)

        print('DONE WITH {}/{}\n '.format(month, year))
    
        desc_all.to_csv('../data/estimated/estimation_statistics.csv')
        network_data_all.to_csv('../data/estimated/network_data.csv')

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   37.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  5.3min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   47.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.9min finished


DONE WITH 1/2001
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


...

DONE WITH 11/2005
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.1min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.5min finished


DONE WITH 12/2005
 
CPU times: user 27min 32s, sys: 7min 21s, total: 34min 53s
Wall time: 7h 41min 46s


In [ ]:
%%time
for year in range(2006, 2011):
    for month in range(1, 13): #[3, 6, 9, 12]
        preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)
    
        # append data
        desc_all = desc_all.append(desc.rename(preprocessed_data.index[-1]).to_frame().T)
        network_data.index = pd.MultiIndex.from_product([[preprocessed_data.index[-1]],network_data.index], names=['sampling_date', 'permno'])
        network_data_all = network_data_all.append(network_data)

        print('DONE WITH {}/{}\n '.format(month, year))
    
        desc_all.to_csv('../data/estimated/estimation_statistics.csv')
        network_data_all.to_csv('../data/estimated/network_data.csv')

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   34.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.6min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   34.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.5min finished


DONE WITH 1/2006
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   52.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.5min finished
/home/fbrun/.local/lib/python3.6/site-packages/sklearn/covariance/_graph_lasso.py:266: ConvergenceWarning: graphical_lasso: did not converge after 200 iteration: dual gap: 5.260e-04
  % (max_iter, d_gap), ConvergenceWarning)


DONE WITH 3/2010
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   34.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  5.2min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   51.3s


In [10]:
%%time
for year in range(2010, 2011):
    for month in range(3, 13): #[3, 6, 9, 12]
        preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)
    
        # append data
        desc_all = desc_all.append(desc.rename(preprocessed_data.index[-1]).to_frame().T)
        network_data.index = pd.MultiIndex.from_product([[preprocessed_data.index[-1]],network_data.index], names=['sampling_date', 'permno'])
        network_data_all = network_data_all.append(network_data)

        print('DONE WITH {}/{}\n '.format(month, year))
    
        desc_all.to_csv('../data/estimated/estimation_statistics.csv')
        network_data_all.to_csv('../data/estimated/network_data.csv')

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   35.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  5.3min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   54.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.6min finished
/home/fbrun/.local/lib/python3.6/site-packages/sklearn/covariance/_graph_lasso.py:266: ConvergenceWarning: graphical_lasso: did not converge after 200 iteration: dual gap: 5.260e-04
  % (max_iter, d_gap), ConvergenceWarning)


DONE WITH 3/2010
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


...

DONE WITH 11/2010
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.1min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   43.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.4min finished


DONE WITH 12/2010
 
CPU times: user 5min 33s, sys: 1min 44s, total: 7min 17s
Wall time: 1h 33min 18s


In [ ]:
%%time
for year in range(2011, 2016):
    for month in range(1, 13): #[3, 6, 9, 12]
        preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)
    
        # append data
        desc_all = desc_all.append(desc.rename(preprocessed_data.index[-1]).to_frame().T)
        network_data.index = pd.MultiIndex.from_product([[preprocessed_data.index[-1]],network_data.index], names=['sampling_date', 'permno'])
        network_data_all = network_data_all.append(network_data)

        print('DONE WITH {}/{}\n '.format(month, year))
    
        desc_all.to_csv('../data/estimated/estimation_statistics.csv')
        network_data_all.to_csv('../data/estimated/network_data.csv')

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.3min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   41.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.6min finished


DONE WITH 1/2011
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.3min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   58.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.7min finished


DONE WITH 9/2011
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [9]:
%%time
for year in range(2011, 2012):
    for month in range(10, 13): #[3, 6, 9, 12]
        preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)
    
        # append data
        desc_all = desc_all.append(desc.rename(preprocessed_data.index[-1]).to_frame().T)
        network_data.index = pd.MultiIndex.from_product([[preprocessed_data.index[-1]],network_data.index], names=['sampling_date', 'permno'])
        network_data_all = network_data_all.append(network_data)

        print('DONE WITH {}/{}\n '.format(month, year))
    
        desc_all.to_csv('../data/estimated/estimation_statistics.csv')
        network_data_all.to_csv('../data/estimated/network_data.csv')

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.1min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.8min finished
/home/fbrun/.local/lib/python3.6/site-packages/sklearn/covariance/_graph_lasso.py:266: ConvergenceWarning: graphical_lasso: did not converge after 200 iteration: dual gap: -3.857e-04
  % (max_iter, d_gap), ConvergenceWarning)


DONE WITH 10/2011
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


...

DONE WITH 11/2011
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.8min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   51.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.3min finished


DONE WITH 12/2011
 
CPU times: user 1min 21s, sys: 27 s, total: 1min 48s
Wall time: 28min 22s


In [10]:
%%time
for year in range(2012, 2016):
    for month in range(1, 13): #[3, 6, 9, 12]
        preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)
    
        # append data
        desc_all = desc_all.append(desc.rename(preprocessed_data.index[-1]).to_frame().T)
        network_data.index = pd.MultiIndex.from_product([[preprocessed_data.index[-1]],network_data.index], names=['sampling_date', 'permno'])
        network_data_all = network_data_all.append(network_data)

        print('DONE WITH {}/{}\n '.format(month, year))
    
        desc_all.to_csv('../data/estimated/estimation_statistics.csv')
        network_data_all.to_csv('../data/estimated/network_data.csv')

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.3min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   53.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.7min finished


DONE WITH 1/2012
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


...

DONE WITH 11/2015
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.0min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   40.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.6min finished


DONE WITH 12/2015
 
CPU times: user 20min 52s, sys: 5min 51s, total: 26min 44s
Wall time: 6h 9min 23s


In [ ]:
%%time
for year in range(2016, 2021):
    for month in range(1, 13): #[3, 6, 9, 12]
        preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)
    
        # append data
        desc_all = desc_all.append(desc.rename(preprocessed_data.index[-1]).to_frame().T)
        network_data.index = pd.MultiIndex.from_product([[preprocessed_data.index[-1]],network_data.index], names=['sampling_date', 'permno'])
        network_data_all = network_data_all.append(network_data)

        print('DONE WITH {}/{}\n '.format(month, year))
    
        desc_all.to_csv('../data/estimated/estimation_statistics.csv')
        network_data_all.to_csv('../data/estimated/network_data.csv')

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.0min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   43.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.3min finished


DONE WITH 1/2016
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


...

DONE WITH 11/2018
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  3.9min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.8min finished


DONE WITH 12/2018
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...


In [8]:
%%time
for year in range(2019, 2021):
    for month in range(1, 13): #[3, 6, 9, 12]
        preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)
    
        # append data
        desc_all = desc_all.append(desc.rename(preprocessed_data.index[-1]).to_frame().T)
        network_data.index = pd.MultiIndex.from_product([[preprocessed_data.index[-1]],network_data.index], names=['sampling_date', 'permno'])
        network_data_all = network_data_all.append(network_data)

        print('DONE WITH {}/{}\n '.format(month, year))
    
        desc_all.to_csv('../data/estimated/estimation_statistics.csv')
        network_data_all.to_csv('../data/estimated/network_data.csv')

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.0min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   34.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.9min finished
/home/fbrun/.local/lib/python3.6/site-packages/sklearn/covariance/_graph_lasso.py:266: ConvergenceWarning: graphical_lasso: did not converge after 200 iteration: dual gap: -1.885e-04
  % (max_iter, d_gap), ConvergenceWarning)


DONE WITH 1/2019
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


...

DONE WITH 11/2020
 
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.4min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.3min finished
/home/fbrun/.local/lib/python3.6/site-packages/sklearn/covariance/_graph_lasso.py:266: ConvergenceWarning: graphical_lasso: did not converge after 200 iteration: dual gap: -5.112e-03
  % (max_iter, d_gap), ConvergenceWarning)


DONE WITH 12/2020
 
CPU times: user 11min 14s, sys: 3min 17s, total: 14min 32s
Wall time: 3h 8min 33s
